In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import gc

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
df_frame = pd.DataFrame(df)
df_frame_feb = pd.DataFrame(df_feb)
df_frame.rename(columns={'airport_fee': 'Airport_fee'}, inplace=True)

In [4]:
del df,df_feb
gc.collect()

0

In [5]:
df_frame['duration_minutes'] = (df_frame['tpep_dropoff_datetime'] - df_frame['tpep_pickup_datetime']).dt.total_seconds() / 60
df_frame_feb['duration_minutes'] = (df_frame_feb['tpep_dropoff_datetime'] - df_frame_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

In [6]:
std_dev = df_frame['duration_minutes'].std()
print("Standard Deviation of Duration (minutes):", std_dev)

Standard Deviation of Duration (minutes): 42.594351241920904


In [7]:
# Filter for durations between 1 and 60 minutes (inclusive)
df_valid = df_frame[(df_frame['duration_minutes'] >= 1) & (df_frame['duration_minutes'] <= 60)]
df_valid_feb = df_frame_feb[(df_frame_feb['duration_minutes'] >= 1) & (df_frame_feb['duration_minutes'] <= 60)]

In [8]:
fraction = len(df_valid) / len(df_frame)
print("Fraction of valid records:", fraction)

Fraction of valid records: 0.9812202822125979


In [9]:
df_valid.loc[:, 'PULocationID'] = df_valid['PULocationID'].astype(str)
df_valid.loc[:, 'DOLocationID'] = df_valid['DOLocationID'].astype(str)
df_valid_feb.loc[:, 'PULocationID'] = df_valid_feb['PULocationID'].astype(str)
df_valid_feb.loc[:, 'DOLocationID'] = df_valid_feb['DOLocationID'].astype(str)

/tmp/ipykernel_84411/1904111145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid.loc[:, 'PULocationID'] = df_valid['PULocationID'].astype(str)
/tmp/ipykernel_84411/1904111145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid.loc[:, 'DOLocationID'] = df_valid['DOLocationID'].astype(str)
/tmp/ipykernel_84411/1904111145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [10]:
del df_frame,df_frame_feb
gc.collect()

373

In [11]:
list_of_dicts = df_valid[['PULocationID','DOLocationID']].to_dict(orient='records')
list_of_dicts_feb = df_valid_feb[['PULocationID','DOLocationID']].to_dict(orient='records')

In [14]:
# Initialize the DictVectorizer to return sparse matrix
vectorizer = DictVectorizer(sparse=True)

# Fit and transform the list of dictionaries into a sparse matrix
X_sparse = vectorizer.fit_transform(list_of_dicts)
X_sparse_feb = vectorizer.transform(list_of_dicts_feb)

In [13]:
num_columns = X_sparse.shape[1]
print("Number of columns (features):", num_columns)

Number of columns (features): 515


In [15]:
from scipy.sparse import save_npz
save_npz('X_sparse_feb.npz', X_sparse_feb)

In [ ]:
del df_frame,df_frame_feb
gc.collect()

In [16]:
%whos

Variable            Type              Data/Info
-----------------------------------------------
DictVectorizer      type              <class 'sklearn.feature_e<...>ctorizer.DictVectorizer'>
X_sparse            csr_matrix          (0, 43)	1.0\n  (0, 325)<...>1.0\n  (3009172, 435)	1.0
X_sparse_feb        csr_matrix          (0, 67)	1.0\n  (0, 304)<...>1.0\n  (2855950, 325)	1.0
df_valid            DataFrame                  VendorID tpep_pi<...>009173 rows x 20 columns]
df_valid_feb        DataFrame                  VendorID tpep_pi<...>855951 rows x 20 columns]
fraction            float             0.9812202822125979
gc                  module            <module 'gc' (built-in)>
list_of_dicts       list              n=3009173
list_of_dicts_feb   list              n=2855951
np                  module            <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
num_columns         int               515
pd                  module            <module 'pandas' from '/h<...>ages/pandas/__

In [17]:
del list_of_dicts,list_of_dicts_feb
gc.collect()

0

In [18]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Step 1: Prepare the features (X) and response variable (y)
X = X_sparse  # Feature matrix (sparse matrix from previous step)
y = df_valid['duration_minutes']  # Response variable

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
print(f"RMSE on Training Data: {rmse_train}")

RMSE on Training Data: 7.6481236638615435


NameError: name 'joblib' is not defined

In [19]:
import joblib
joblib.dump(model, 'linear_model.pkl')

['linear_model.pkl']

In [20]:
#Validation set
y_feb = df_valid_feb['duration_minutes']
y_valid_pred = model.predict(X_sparse_feb)
mse_valid = mean_squared_error(y_feb, y_valid_pred)
rmse_valid = np.sqrt(mse_valid)
print(f"RMSE on Validation Data: {rmse_valid}")

RMSE on Validation Data: 7.812140339904778
